In [12]:
import subprocess

script_path = "../create_db.sh"

try:
    result = subprocess.run([script_path], check=True, capture_output=True, text=True)
    
except subprocess.CalledProcessError as e:
    print(f"Script failed with error: {e.stderr}")

In [15]:
import psycopg2
import os

# Database connection parameters
db_params = {
    "dbname": "holocron",
    "host": "localhost",  # change when deploying
    "port": "5432"  # usually 5432 for PostgreSQL
}

# Define the path to the SQL file
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sql_file_path = os.path.join(parent_dir, "..", "sql", "001_schema.sql")

# Read SQL commands from the file
with open(sql_file_path, "r", encoding="utf-8") as file:
    sql_commands = file.read()

try:
    # Establish the database connection
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    
    # Execute the SQL commands
    cursor.execute(sql_commands)
    connection.commit()
    
except Exception as error:
    print(f"Error while executing SQL commands: {error}")
    if connection:
        connection.rollback()
    
finally:
    # Close the cursor and connection
    if "cursor" in locals() and cursor:
        cursor.close()
    if "connection" in locals() and connection:
        connection.close()

In [17]:
%reload_ext autoreload
%autoreload 2

# Set parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..", os.pardir))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Import utilities
from utils import extract
from utils import common

# Load the configuration from a file
CONFIG = configparser.ConfigParser()
CONFIG.read("../../config.ini")

# Initialize the OpenAI client with an API key from the configuration
CLIENT = OpenAI(api_key=CONFIG["OPENAI"]["OPENAI_API_KEY"])

# Database connection parameters
db_params = {
    "dbname": "holocron",
    "host": "localhost",
    "port": "5432",
    "user": "your_username",  # Add your username
    "password": "your_password"  # Add your password
}

embedding_model = "text-embedding-ada-002"
conversations_filepath = "../../data/test_inputs/multi-user-test-conversations.jsonl"
conversations = common.read_jsonl_file(conversations_filepath)
how_many_conversations = 1

for conversation in itertools.islice(conversations, how_many_conversations):
    prompt_data = extract.extract_prompt_data(conversation)
    messages = prompt_data["messages"]
    bot_first_name = prompt_data["bot"]["first_name"]
    assistant_message_idxs = [idx for idx, message in enumerate(messages) if message["role"] == "assistant" and message["first_name"] == bot_first_name]

    for idx in assistant_message_idxs:
        message_to_embed = messages[idx - 1]["message"] + " -> " + messages[idx]["message"]
        
        response = CLIENT.embeddings.create(
            input=message_to_embed,
            model=embedding_model
        )
        embedding = response.data[0].embedding

        try:
            connection = psycopg2.connect(**db_params)
            cursor = connection.cursor()
            
            cursor.execute(
                """
                INSERT INTO search.memories (
                    bot_in_channel_id, external_channel_id, 
                    external_identity_id, message_id, message_timestamp, 
                    message_embedding, metadata)
                VALUES (%s, %s, %s, %s, %s, %s::vector, %s)
                """,
                (1, 1, 1, idx, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), embedding, json.dumps({"message": message_to_embed}))
            )
            
            connection.commit()
        except Exception as error:
            print(f"Error while executing SQL commands: {error}")
            if connection:
                connection.rollback()
        finally:
            if cursor:
                cursor.close()
            if connection:
                connection.close()

In [22]:
import psycopg2
import os

# Database connection parameters
db_params = {
    "dbname": "holocron",
    "host": "localhost",  # change when deploying
    "port": "5432"  # usually 5432 for PostgreSQL
}

# Define the path to the SQL file
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sql_file_path = os.path.join(parent_dir, "..", "sql", "001_schema.sql")

try:
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    # Assume you have a query message to search for
    query_message = "your query message"
    response = CLIENT.embeddings.create(
        input=query_message,
        model=embedding_model
    )
    query_embedding = response.data[0].embedding

    # Execute the SQL commands
    cursor.execute(
        """
        SELECT metadata, message_embedding <-> %s::vector AS distance
        FROM search.memories
        ORDER BY distance
        LIMIT 5
        """,
        (query_embedding,)
    )

    results = cursor.fetchall()
    connection.commit()
    
    print(results)
    
except Exception as error:
    print(f"Error while executing SQL commands: {error}")
    if connection:
        connection.rollback()
    
finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Error while executing SQL commands: operator does not exist: jsonb <-> vector
LINE 2:         SELECT metadata, message_embedding <-> ARRAY[ -0.014...
                                                   ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

